In [4]:
# https://markhedleyjones.com/projects/python-tone-generator

In [1]:
import math
import time

import numpy
import pyaudio


def sine(freq, length, rate):
    length = int(length * rate)
    factor = float(freq) * (math.pi * 2) / rate
    return numpy.sin(numpy.arange(length) * factor)


class ToneGenerator(object):
    def __init__(self, samplerate=44100, frames_per_buffer=4410):
        self.p = pyaudio.PyAudio()
        self.samplerate = samplerate
        self.frames_per_buffer = frames_per_buffer
        self.streamOpen = False
        self.running = False
        self.frequency = 0

    def stop(self):
        self.running = False

    def _gen_chunk(self):
        chunks = [sine(self.frequency, self.frames_per_buffer, 100)]
        chunk = numpy.concatenate(chunks) * 0.25

        fade = 50.0
        fade_in = numpy.arange(0.0, 1.0, 1 / fade)
        fade_out = numpy.arange(1.0, 0.0, -1 / fade)
        chunk[:fade] = numpy.multiply(chunk[:fade], fade_in)
        chunk[fade:] = numpy.multiply(chunk[fade:], fade_out)
        return chunk

    def callback(self, in_data, frame_count, time_info, status):
        if self.running:
            data = self._gen_chunk().astype(numpy.float32)
            return (data.tobytes(), pyaudio.paContinue)
        else:
            return (None, pyaudio.paComplete)

    def is_playing(self):
        if self.stream.is_active():
            return True
        else:
            if self.streamOpen:
                self.stream.stop_stream()
                self.stream.close()
                self.streamOpen = False
            return False

    def setNote(self, frequency, amplitude):
        self.frequency = frequency
        self.amplitude = amplitude

    def play(self):
        self.running = True
        self.omega = float(0.5) * (math.pi * 2) / self.samplerate
        self.amplitude = 0
        self.buffer_offset = 0
        self.streamOpen = True
        self.stream = self.p.open(
            format=pyaudio.paFloat32,
            channels=1,
            rate=self.samplerate,
            output=True,
            frames_per_buffer=self.frames_per_buffer,
            stream_callback=self.callback,
        )

In [2]:
notes = [130.77, 111.17, 140.19, 121.98, 146.85]
amplitude = 0.8
step_duration = 0.4
index = 0
generator = ToneGenerator()
generator.play()

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
Traceback (most recent call last):
  File "/tmp/ipykernel_94666/3121833986.py", line 39, in callback
    data = self._gen_chunk().astype(numpy.float32)
  

In [ ]:
while generator.is_playing():
    frequency = notes[index % len(notes)]
    generator.setNote(frequency, amplitude)
    index += 1
    time.sleep(0.7)

In [2]:
generator.stop()

NameError: name 'generator' is not defined

In [ ]:
xs = numpy.arange(
    generator.buffer_offset, generator.buffer_offset + generator.frames_per_buffer
)
out = generator.amplitude * numpy.sin(xs * generator.omega)

In [10]:
out

array([-0.59339342, -0.60328659, -0.61297034, ..., -0.76254185,
       -0.76691677, -0.77102547])

In [11]:
xs

array([1009890, 1009891, 1009892, ..., 1014297, 1014298, 1014299])

In [22]:
from matplotlib import pyplot as plt

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>